<a href="https://colab.research.google.com/github/arun-arunisto/GiftoLexia/blob/main/GiftoLexia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
data_file = pd.read_csv("/content/drive/MyDrive/Datascience&MachineLearning/datasets/org_data.csv")
data_file.head()

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o
1,10,0,Dena Cremin,production manager
2,20,0,Oda Stiedemann,marketing manager
3,30,10,Ashley Davis,plant manager (fabrication)
4,40,10,Ayana Greenfelder MD,plant manager(assembly)


In [4]:
emp_id = data_file["id"]
emp_id.head()

0     0
1    10
2    20
3    30
4    40
Name: id, dtype: int64

In [5]:
data = data_file[data_file["id"] == 0]
data

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o


In [6]:
#finding the employee data with given id
def fetch_emp_data(data, id):
  emp = data[data["id"] == id]
  return emp

In [7]:
emp_data = fetch_emp_data(data_file, 10)
emp_data

,id,manager_id,name,designation
1,10,0,Dena Cremin,production manager


In [8]:
#finding employee reporting to the employee id
def fetch_emp_reporting(data, id):
  rep_emp = data[data["manager_id"]==id]
  return rep_emp

In [9]:
rep_emp = fetch_emp_reporting(data_file, 0)
rep_emp

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o
1,10,0,Dena Cremin,production manager
2,20,0,Oda Stiedemann,marketing manager


In [10]:
rep_emp["id"]

0     0
1    10
2    20
Name: id, dtype: int64

In [11]:
#creating a new data frame
new_df = pd.DataFrame(columns=["id", "manager_id", "name", "designation"])
new_df

,id,manager_id,name,designation


In [12]:
#adding data to new data frame
new_df = pd.concat([new_df, rep_emp])
new_df

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o
1,10,0,Dena Cremin,production manager
2,20,0,Oda Stiedemann,marketing manager


In [13]:
#dataframe filter using lambda
filtered_data = data_file[data_file.apply(lambda row: row["id"] in [0, 10], axis=1)]
filtered_data

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o
1,10,0,Dena Cremin,production manager


In [14]:
REPORT_PERMISSION = {
	0:(10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160),
	10:(30, 40, 50, 60, 70, 80, 90, 100),
	20:(110, 120, 130, 140, 150, 160)
}

In [16]:
filtered_data = data_file[data_file["id"]==0]
filtered_data

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o


In [23]:
report_per = REPORT_PERMISSION[20]
report_per

(110, 120, 130, 140, 150, 160)

In [24]:
filtered_data = data_file[data_file.apply(lambda row: row["id"] in report_per, axis=1)]
filtered_data

,id,manager_id,name,designation
11,110,20,Rod Hoppe,sales head (Chennai region)
12,120,20,Veda Bosco,sales head (Bangalore region)
13,130,110,Ray Grimes,sales officer (Chennai region)
14,140,110,Dayana Dicki,sales officer (Chennai region)
15,150,120,Miss Zita Williamson,sales officer (Bangalore region)
16,160,120,Maudie Runte,sales officer (Bangalore region)


In [ ]:
def get_employee_hierarchy(employee_id, data):
    # Find the employee record with the given ID
    employee = next((emp for emp in data if emp[0] == employee_id), None)
    if employee is None:
        return []

    # Add the current employee to the result
    result = [employee]

    # Find all employees reporting to the current employee
    subordinates = [emp for emp in data if emp[1] == employee_id]

    # Recursively get hierarchy for each subordinate
    for subordinate in subordinates:
        result.extend(get_employee_hierarchy(subordinate[0], data))

    return result

def filter_employee_data(employee_id, data):
    # Get the hierarchy of employees starting from the given employee ID
    hierarchy = get_employee_hierarchy(employee_id, data)

    # Extract the IDs of employees in the hierarchy
    hierarchy_ids = set(emp[0] for emp in hierarchy)

    # Filter the data to include only the employees in the hierarchy
    filtered_data = [emp for emp in data if emp[0] in hierarchy_ids]

    return filtered_data

In [38]:
employee = data_file[data_file["id"]==0]
employee

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o


In [39]:
result = pd.concat([pd.DataFrame(), employee])
result

,id,manager_id,name,designation
0,0,0,Jarrell Rau,c.e.o


In [44]:
subordinates = data_file[data_file["manager_id"]==160]
subordinates.empty

True

In [41]:
for _, row in subordinates.iterrows():
  print(row)

id                       0
manager_id               0
name           Jarrell Rau
designation          c.e.o
Name: 0, dtype: object
id                             10
manager_id                      0
name                  Dena Cremin
designation    production manager
Name: 1, dtype: object
id                            20
manager_id                     0
name              Oda Stiedemann
designation    marketing manager
Name: 2, dtype: object


In [61]:
def get_employee_hierarchy(data, emp_id):
  if emp_id == 0:
    return data
  employee = data[data["id"]==emp_id]
  if employee is None:
    return pd.DataFrame()
  result = pd.concat([pd.DataFrame(), employee])
  subordinates = data[data["manager_id"]==emp_id]
  if subordinates.empty:
    return result
  for _, row in subordinates.iterrows():
    result = pd.concat([result, get_employee_hierarchy(data, row["id"])])
  return result

In [62]:
def filter_employee_data(data, emp_id):
  hierarchy = get_employee_hierarchy(data, emp_id)
  return hierarchy

In [69]:
result = filter_employee_data(data_file, 70)
result

,id,manager_id,name,designation
7,70,50,Erik Hand,worker (fabrication)
